# Week 1

This problem sheet tests the representation of numbers on the computer, using
modular arithmetic. We also use floating point rounding modes to implement 
interval arithmetic, and thereby
produce rigorous bounds on the exponential.

In [1]:
using ColorBitstring, SetRounding

## 1. Binary representation

**Problem 1.1** Like decimal representation, all rational numbers have repeating sequence of binary digits,
that is, $b_k = b_{k+r}$ for fixed integers $r$ and $K$ and all $k \geq K$. Use the geometric series
to determine the corresponding real number, and thence prove that a repeating sequence of binary digits
implies that a number is rational.

**Problem 1.2** What are the binary representations of $1/5$ and $1/100$? 
What is $\pi$ to 5 binary decimal places? Hint: recall that $\pi \approx  3.14$.


## 2. Integers

**Problem 2.1** With 8-bit signed integers, find the bits for the following:
$$
10, 120, -10
$$
**SOLUTION**
We can find the binary digits by repeatedly subtracting the largest power of 2 less than a number
until we reach 0, e.g.:
$$
10 -2^3 - 2 = 0
$$
implies
$$
10 = (101)_2
$$
Thus the bits are:

In [2]:
printlnbits(Int8(10))

00001010


Similarly,
$$
120 = 2^6 + 2^5 + 2^4 + 2^3 = (1111000)_2
$$
Thus the bits are:

In [3]:
printlnbits(Int8(120))

01111000


For negative numbers we perform the same trick but adding $2^p$ to make it positive, e.g.,
$$
-10 = 2^8 - 10 ({\rm mod 2^8}) = 246 = 2^7 + 2^6 + 2^5 + 2^4 + 2^2 + 2 = (11110110)_2
$$
This the bits are:

In [4]:
printlnbits(Int8(-10))

11110110


**Problem 2.2** What will `Int8(120) + Int8(10)` return?

**SOLUTION**
It will return
$$
130 ({\rm mod\ } 2^8) = -126 ({\rm mod\ } 2^8)
$$

In [5]:
Int8(120) + Int8(10)

-126

## 3. Floating point numbers

**Problem 3.1** What is the single precision $F_{32}$ (`Float32`) floating point representation for the following: 
$$
2, 30, 31, 32, 33, 23/4, (23/4)\times 2^{100}, (23/4)\times 2^{−100}, (23/4)\times2^{−135}
$$
When rounded to nearest, what are the single precision bits of $1/5$?
Check your answers using `printbits`.


**SOLUTION**
Recall that we have `σ,Q,S = 127,8,23`. Thus we write
$$
2 = 2^{128-127} * (1.00000000000000000000000)_2
$$
The exponent bits are those of
$$
128 = 2^7 = (10000000)_2
$$
Hence we get

In [6]:
printlnbits(2f0)

01000000000000000000000000000000


In [7]:
printlnbits(30f0)
printlnbits(31f0)
printlnbits(32f0)
printlnbits(33f0)
printlnbits(23f0/4)
printlnbits(2f0^100)
printlnbits(23f0/4 * 2f0^(-100))
printlnbits(23f0/4 * 2f0^(-135))
printlnbits(1f0/5)

01000001111100000000000000000000
01000001111110000000000000000000
01000010000000000000000000000000
01000010000001000000000000000000
01000000101110000000000000000000
01110001100000000000000000000000
00001110101110000000000000000000
00000000000000010111000000000000
00111110010011001100110011001101


**Problem 3.2** Let $m(y) = \min\{x \in F_{32} : x > y \}$ be the smallest single precision number
greater than $y$. What is $m(2) - 2$ and $m(1024) - 1024$? Check your answer using the `nextfloat`
command.

In [8]:
nextfloat(2f0) - 2, nextfloat(1024f0) - 1024

(2.3841858f-7, 0.00012207031f0)

**SOLUTION**

## 4. Arithmetic


**Problem 4.1** Suppose $x = 1.25$ and consider 16-bit floating point arithmetic (`Float16`). 
What is the error in approximating $x$ by the nearest float point number ${\rm fl}(x)$?
What is the error in approximating $2x$, $x/2$, $x + 2$ and $x - 2$ by $2 \otimes x$, $x \oslash 2$, $x \oplus 2$ and $x \ominus 2$?
For what floating point numbers is $x \oslash 2 \neq x/2$ and $x \oplus 2 \neq x + 2$?

**Problem 4.2** Explain the following:

In [9]:
x = 10.0^100
x + 1 == x

true

What is the largest floating point number such that `x + 1 ≠ x`?

**Problem 4.3** What are the exact bits for $1/5$, $1/5 + 1$, $(1/5)^2$, and $(1.1-1)/0.1$  computed
using  half-precision (`Float16`) (using
default rounding)?

**Problem 4.4** Find a bound on the floating point error for the following
$$
\begin{align*}
(1.1 ⊖ 1) &⊘ 0.1 \\
(1.1 ⊕ 1.2) &⊗ 1.3
\end{align*}
$$
How does your bound compare with the true error?



## 5. Implementation



**Problem 5.1** Complete the following function which computes the first `n` terms of the Taylor series of $\exp(x)$, that is,
$$
\sum_{k=0}^n {x^k \over k!}
$$
using only algebraic operations (e.g. do not call `factorial(k)`).

In [10]:
function exp_t(x, n)
    ret = one(x) # 1 of same type as x
    s = one(x)
    for k = 1:n
        s = s/k * x
        ret = ret + s
    end
    ret
end

exp_t (generic function with 1 method)

The following problems consider implementation of interval arithmetic for
proving precise bounds on arithmetic operations. That is recall the set operations
$$
A + B = \{x + y : x \in A, y \in B\}, AB = \{xy : x \in A, y \in B\}.
$$
We want to implement floating point variants such that, for $S = [a,b] + [c,d]$
and $P = [a,b] * [c,d]$,
$$
\begin{align*}
[a,b] \oplus [c,d] &:= [{\rm fl}^{\rm down}(\min S), {\rm fl}^{\rm up}(\max S)] \\
[a,b] \oplus [c,d] &:= [{\rm fl}^{\rm down}(\min P), {\rm fl}^{\rm up}(\max P)]
\end{align*}
$$
This guarantees $[a,b] + [c,d] \subset [a,b] \oplus [c,d]$, i.e., if $x \in [a,b]$ and
$y \in [c,d]$ then $x +y \in [a,b] \oplus [c,d]$, and we thereby have rigorous bounds.


**Problem 5.2** For intervals $A = [a,b]$ and $B = [c,d]$ such that $0 \notin A,B$
 and integer $n \neq 0$, 
deduce formulas for the minimum and maximum of $A/n$, $A+B$ and $AB$.

**Solution**

$$
\begin{align*}
{A \over n} &= \begin{cases}
[a/n,b/n] & n > 0 \\
[b/n,a/n] & n < 0
\end{cases},\\
A + B &= [a + c, b + d] \\
AB &= \begin{cases}
[cd,ab]& a,b,c,d < 0 \\
[ad,bc]& a,b < 0 \hbox{ and } c,d >0 \\
[bc,ad]& a,b > 0 \hbox{ and } c,d  < 0 \\
[ab,cd]& a,b,c,d >> 0
\end{cases}
\end{align*}
$$




**Problem 5.3** Use the formulae from Problem 5.2 to complete the following definition:

In [11]:
struct Interval{T}
    a::T
    b::T
end

import Base: *, +, -, /, one, in
one(x::Interval) = Interval(one(x.a), one(x.b))
in(x, y::Interval) = y.a ≤ x ≤ y.b

function /(x::Interval, n::Integer)
    T = typeof(x.a)
    if iszero(n)
        error("Dividing by zero not support")
    end
    a = setrounding(T, RoundDown) do
        # TODO: lower bound
        ## SOLUTION
        if n > 0
            x.a / n
        else
            x.b / n
        end
        ## END
    end
    b = setrounding(T, RoundUp) do
        # TODO: upper bound
        ## SOLUTION
        if n > 0
            x.b / n
        else
            x.a / n
        end
        ## END
    end
    Interval(a, b)
end

function *(x::Interval, y::Interval)
    T = promote_type(typeof(x.a), typeof(x.b))
    if 0 in x || 0 in y
        error("Multiplying with intervals containing 0 not supported.")
    end
    a = setrounding(T, RoundDown) do
        # TODO: lower bound
        ## SOLUTION
        if x.a < 0 && x.b < 0 && y.a < 0 && y.b < 0
            y.b * x.b
        elseif x.a < 0 && x.b < 0 && y.a > 0 && y.b > 0
            x.a * y.b
        elseif x.a > 0 && x.b > 0 && y.a < 0 && y.b < 0
            x.b * y.a
        else
            x.a * y.a
        end
        ## END
    end
    b = setrounding(T, RoundUp) do
        # TODO: upper bound
        ## SOLUTION
        if x.a < 0 && x.b < 0 && y.a < 0 && y.b < 0
            y.a * x.a
        elseif x.a < 0 && x.b < 0 && y.a > 0 && y.b > 0
            x.b * y.a
        elseif x.a > 0 && x.b > 0 && y.a < 0 && y.b < 0
            x.a * y.b
        else
            x.b * y.b
        end
        ## END
    end
    Interval(a, b)
end

function +(x::Interval, y::Interval)
    T = promote_type(typeof(x.a), typeof(x.b))
    a = setrounding(T, RoundDown) do
        # TODO: upper bound
        ## SOLUTION
        x.a + y.a
        ## END
    end
    b = setrounding(T, RoundUp) do
        # TODO: upper bound
        ## SOLUTION
        x.b + y.b
        ## END
    end
    Interval(a, b)
end

+ (generic function with 247 methods)

**Problem 5.4** Bound the tail of the Taylor series for ${\rm e}^x$ assuming $|x| \leq 1$. 
(Hint: ${\rm e}^x \leq 3$ for $x \leq 1$.)
Use the bound
to complete the function `exp_bound` which computes ${\rm e}^x$ with rigorous error bounds, that is
so that when applied to an interval $[a,b]$ it returns an interval that is 
guaranteed to contain the interval $[{\rm e}^a, {\rm e}^b]$.

In [12]:
function exp_bound(x::Interval, n)
    if abs(x.a) > 1 || abs(x.b) > 1
        error("Interval must be a subset of [-1, 1]")
    end
    ret = exp_t(x, n) # the code for Taylor series should work on Interval unmodified
    f = factorial(min(20, n + 1)) # avoid overflow in computing factorial
    T = typeof(ret.a)

    # TODO: modify ret so that exp(x) is guaranteed to lie in it
    ## SOLUTION
    err = setrounding(T, RoundUp) do
        3 / f
    end
    ret + Interval(-err,err)
    ## END
end

exp_bound (generic function with 1 method)

Check your result by assuring that
the following returns true:

In [13]:
exp(big(1)) in exp_bound(Interval(1.0,1.0), 20) && exp(big(-1)) in exp_bound(Interval(-1.0,-1.0), 20)

true

**SOLUTION** From the Taylor remainder theorem we know the error is
$$
{f^{(k+1)}(ξ) \over (k+1)!} |x|^{k+1} \leq {3 \over (k+1)!}
$$
Thus by widening the computation by this error we ensure that we have
captured the error by truncating the Taylor series.


**Problem 5.4 (advanced)**  Compute the first 1000 (decimal) digits of ℯ, with rigorous error bounds.
Check your result versus `exp(big(1.0))`.

**SOLUTION**
The tricky part of this problem is that we need to fix the bound on `factorial(n)` above
as otherwise we won't get the necessary accuracy. We can do this as follows:
as follows:

In [14]:
function exp_bound(x::Interval, n)
    if abs(x.a) > 1 || abs(x.b) > 1
        error("Interval must be a subset of [-1, 1]")
    end
    ret = exp_t(x, n) # the code for Taylor series should work on Interval unmodified
    T = typeof(ret.a)
    err = setrounding(T, RoundUp) do
        err_i = 3one(T)
        for k = 1:n+1
            err_i = err_i/k
        end
        err_i
    end
    ret + Interval(-err,err)
end
setprecision(5000) do 
    x = Interval(big(1.0),big(1.0))
    exp_bound(x, 600).a
end

2.71828182845904523536028747135266249775724709369995957496696762772407663035354759457138217852516642742746639193200305992181741359662904357290033429526059563073813232862794349076323382988075319525101901157383418793070215408914993488416750924476146066808226480016847741185374234544243710753907774499206955170276183860626133138458300075204493382656029760673711320070932870912744374704723069697720931014169283681902551510865746377211125238978442505695369677078544996996794686445490598793163688923009879312773617821542499922957635148220826989519366803318252886939849646510582093923982948879332036250944311730123819706841614039701983767932068328237646480429531180232878250981945581530175671736133206981125099618188159304169035159888851934580727386673858942287922849989208680582574927961048419844436346324496848756023362482704197862320900216099023530436994184914631409343173814364054625315209618369088870701676839642437814059271456354906130310720851038375051011574770417189861068739696552126715468895703503

The precision and number of Taylor series were increased until the
bounsa matched `exp(big(1.0))` to over 1000 digits:

In [15]:
setprecision(5000) do 
    x = Interval(big(1.0),big(1.0))
    max(exp(big(1.0)) - exp_bound(x, 600).a, exp_bound(x, 600).b - exp(big(1.0)))
end

5.26113165708542304461350380379002020807829172074535961727555539762554502762374303238460123570685421169143172309691861593530015027972940988238875875362809141096197014933452863423527172358220042318698508245539366397434385644453236817189178803648331564991936094005529953966998446212155125033569704852763450898800525304370736736804231040203893946445142734122365376502970203488361093713501445060801481200610538009476490344582452684488214477343180947752986781697438731622247940717937292148249400425522748237088873807453664037434296743456369745171976077141876143420416758530073640803643992843529583890615156877378155347318410719136870243159040745027949570625134208053259364909335886339342031035942112012882249695515353107375624367521209186908491756423849454561817495544599461851927932774767920869148163991351677548879629538864010476219508452630366389952536378452181511779600169943369992236066950959373286673770564584128023320930136022835223297929325926445086076306515385444549569576159161728352569542163479